# Determining the optimal number of hidden layers and neurons for an ANN  

- Start Simple
- Grid/Random Search 
- Cross Validation
- Heuristics and Rule of Thumb 
    1. The no. of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
    2. A common practice is to strat with 1-2 hidden layers.

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [18]:
data = pd.read_csv("Churn_Modelling.csv")
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [19]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

one_hot_encoder_geo = OneHotEncoder()
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']])

geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns = one_hot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

## save pre-processings encoders 

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hot_encoder_geo,file)

X= data.drop('Exited',axis=1)
y = data['Exited']

# Split the data in training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# save the scaler
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [24]:
### Define a func to create a model and try different parameters using KerasClassifier

def create_model(neurons=32,layers=1):
    model = Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))   #Sigmoid for binary classification
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])

    return model

In [25]:
##create a keras classifier

model = KerasClassifier(build_fn=create_model,epochs=50,batch_size=10,verbose=0,neurons=32,layers=1)

In [26]:
## Define grid search parameters

param_grid = {
    'neurons':[16,32,64,128],
    'layers':[1,2],
   #'batch_size':[10,20],
    'epochs':[50,100]
}

In [29]:
#Perform Grid Search

grid = GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)
grid_result = grid.fit(X_train,y_train)

# Print the best parameters

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

c:\Users\Dell\python\NLP\ANN-classification\annvenv\ann_venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Best: 0.858125 using {'epochs': 100, 'layers': 1, 'neurons': 32}


# End!